# Task-1

In [13]:
import numpy as np
seed_value=3105
n=320
rng = np.random.default_rng(seed_value)
routes=["NYC-LAX", "LHR-JFK", "SFO-SEA", "DXB-SIN", "MAD-ROM"]
days=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
classes=["economy", "premium", "business"]
route_adj = [140, 220, 60, 180, 80]
class_adj = [0, 80, 220]


In [14]:
tickets = []
for i in range(1, n+1):
    ticket_id = f"T{seed_value}-{i:04d}"   
    route = routes[(i + seed_value) % 5]     
    day = days[(i + seed_value) % 7]
    days_to_departure = 1 + ((i * 3 + seed_value) % 60)
    class_ticket = classes[(i * 2 + seed_value) % 3]

    base = 120 + (days_to_departure * -1.5)
    noise = rng.normal(0, 25)

    price_usd = round(base + route_adj[(i + seed_value) % 5] + class_adj[(i * 2 + seed_value) % 3] + noise,2)
    if i % 28 == 0:
        price_usd = ""

    if i % 45 == 0 and price_usd != "":
        price_usd *= -1

    if i % 37 == 0:
        class_ticket = class_ticket.upper()

    ticket = {
        "ticket_id": ticket_id,
        "route": route,
        "day": day,
        "days_to_departure": days_to_departure,
        "class_tickets": class_ticket,
        "price_usd": price_usd
    }

    tickets.append(ticket)
    


In [15]:
print(len(tickets))   
print(tickets[:5])

320
[{'ticket_id': 'T3105-0001', 'route': 'LHR-JFK', 'day': 'Sat', 'days_to_departure': 49, 'class_tickets': 'business', 'price_usd': 482.46}, {'ticket_id': 'T3105-0002', 'route': 'SFO-SEA', 'day': 'Sun', 'days_to_departure': 52, 'class_tickets': 'premium', 'price_usd': 178.05}, {'ticket_id': 'T3105-0003', 'route': 'DXB-SIN', 'day': 'Mon', 'days_to_departure': 55, 'class_tickets': 'economy', 'price_usd': 200.48}, {'ticket_id': 'T3105-0004', 'route': 'MAD-ROM', 'day': 'Tue', 'days_to_departure': 58, 'class_tickets': 'business', 'price_usd': 317.89}, {'ticket_id': 'T3105-0005', 'route': 'NYC-LAX', 'day': 'Wed', 'days_to_departure': 1, 'class_tickets': 'premium', 'price_usd': 336.62}]


# Task-2

In [16]:
cleaned_tickets=[]
for t in tickets:
    p=t["price_usd"]
    if p=="" or not isinstance(p,(int,float)) or  p<0:
        continue    
    t["class_tickets"] = t["class_tickets"].lower()

    cleaned_tickets.append(t)


In [17]:
print(len(cleaned_tickets))

302


In [18]:
print(cleaned_tickets[:2])

[{'ticket_id': 'T3105-0001', 'route': 'LHR-JFK', 'day': 'Sat', 'days_to_departure': 49, 'class_tickets': 'business', 'price_usd': 482.46}, {'ticket_id': 'T3105-0002', 'route': 'SFO-SEA', 'day': 'Sun', 'days_to_departure': 52, 'class_tickets': 'premium', 'price_usd': 178.05}]


# Task-3

In [52]:
prices=np.array([t["price_usd"] for t in cleaned_tickets],dtype=float)
day_arr=np.array([t["day"] for t in cleaned_tickets])

In [53]:
price_mean=np.mean(prices)
price_std=np.std(prices)

In [54]:
price_mean

np.float64(314.0893377483444)

In [55]:
price_std

np.float64(116.44763885224727)

In [56]:
unique_days, day_index = np.unique(day_arr, return_inverse=True)

In [59]:
count_per_day=np.bincount(day_index)

In [68]:
count_per_day

array([33, 45, 45, 45, 44, 45, 45])

In [60]:
revenue_per_day=np.bincount(day_index,weights=prices)

In [67]:
revenue_per_day

array([10427.96, 13636.03, 14277.35, 13978.46, 13731.21, 14506.07,
       14297.9 ])

In [61]:
total_revenue=prices.sum()

In [73]:
print(total_revenue)

94854.98000000001


In [62]:
daily_sum=revenue_per_day.sum()

In [72]:
print(daily_sum)

94854.98000000001


In [64]:
print(np.isclose(total_revenue, daily_sum))

True


# Task-4

In [74]:
threshold=np.percentile(prices,90)

In [76]:
high_price_tickets=prices[prices>=threshold]

In [82]:
high_price_count=np.sum(high_price_tickets)

In [83]:
print(np.all(high_price_tickets>=threshold))

True


# Task-5

In [84]:
report={
    "total_tickets":len(tickets),
    "cleaned_tickets":len(cleaned_tickets),
    "mean_price":price_mean,
    "std_price":price_std,
    "daily_totals":dict(zip(unique_days,revenue_per_day)),
    "high_price_count":high_price_count
}

In [90]:
validation= np.isclose(prices.sum(), revenue_per_day.sum())

In [93]:
print(f"""

Total tickets: {report['total_tickets']}
Cleaned tickets: {report['cleaned_tickets']}

Average price: ${report['mean_price']}
Price std dev: ${report['std_price']}

Daily revenue:
""")

for day, val in report["daily_totals"].items():
    print(f"  {day}: ${val}")

print(f"""
High-price tickets (top 10%): {report['high_price_count']}
Validation:
Revenue check: {validation}

""")



Total tickets: 320
Cleaned tickets: 302

Average price: $314.0893377483444
Price std dev: $116.44763885224727

Daily revenue:

  Fri: $10427.960000000001
  Mon: $13636.030000000002
  Sat: $14277.350000000004
  Sun: $13978.46
  Thu: $13731.209999999997
  Tue: $14506.070000000002
  Wed: $14297.900000000001

High-price tickets (top 10%): 16157.75
Validation:
Revenue check: True


